In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(precision=3)

import ksource_py as ks
import mcpl

In [ ]:
%matplotlib inline

# Generar muestras artificiales

In [ ]:
N = int(1E5)

pt = "n"
su = 1
Es = np.exp(np.random.normal(0, 1, (N,1)))
sx = sy = 10
poss = np.random.normal([0,0,0], [10,10,0], (N,3))
sdx = sdy = .1
dirs = np.random.normal([0,0,0], [.1,.1,0], (N,3))
dirs[:,2] = np.sqrt(1-dirs[:,0]**2-dirs[:,1]**2)
parts = np.concatenate((Es,poss,dirs), axis=1)
ws = np.random.normal(1, 0.1, N)

tracksfile = "samples.ssv" # "tracks.ssv"
ks.savessv(pt, parts, ws, tracksfile)
mcplfile = ks.convert2mcpl(tracksfile, "ssv")

# Crear KSource

In [ ]:
J = 1
set_params = True
plist = ks.PList(mcplfile, set_params=set_params)
geom = ks.Geometry([ks.Lethargy(1), ks.SurfXY(0), ks.Isotrop()])
s = ks.KSource(plist, geom, bw="silv", J=J)

In [ ]:
# Ajustar KSource

N = -1 # Cantidad de particulas a utilizar
s.bw_method = "silv" # silv, mlcv, knn
s.fit(N)
s.bw /= 5 # Para graficos integrales puede usarse una bw mas baja

# Evaluar estadistica

In [ ]:
# Con MCPL

mcpl.dump_stats(mcpl.collect_stats(mcplfile))
mcpl.plot_stats(mcplfile)

In [ ]:
# Con ks.Stats

# Intervalos de variables (box)
dxymax = -1
dzmin = np.sqrt(1-dxymax**2)
maskfun = ks.Box([None,None,None,None,None,None,dzmin], None)

# Funcion de peso
fact_dosim = ks.H10(pt=plist.pt)
weightfun = lambda parts: parts[:,0]

parts,ws = plist.get()
stats = ks.Stats(parts, ws, weightfun=weightfun, maskfun=maskfun)

N,I,err = stats.mean_weight(steps=100)
plt.show()
N,mn,err = stats.mean(var=1, steps=100)
plt.show()
N,mn,err = stats.std(var=1, steps=100)
plt.show()

# Crear graficos

In [ ]:
# Graficos en E

vec0 = None
vec1 = None
EE = np.logspace(-1,1,100)
fig,scores = s.plot_E(EE, vec0, vec1)

uu = np.log(1/EE)
plt.plot(EE, 1/EE * np.exp(-uu**2/(2*su**2))/(su*np.sqrt(2*np.pi)), zorder=3, label="True")
plt.legend()
plt.show()

In [ ]:
# Graficos en x

vec0 = None # [0, -30,-30, 0,-180]
vec1 = None # [25, 30,30, 1,180]
xx = np.linspace(-30,30,30)
fig,scores = s.plot_integr(xx, "x", vec0, vec1)

pdf = np.exp(-(xx/sx)**2/2)/(sx*np.sqrt(2*np.pi))
plt.plot(xx, pdf, zorder=5, label="True")
plt.legend()
plt.show()


In [ ]:
# Graficos en dz

vec0 = None # [20, -30,-30, 0,-180]
vec1 = None # [25, 30,30, 1,180]
ddz = np.linspace(0.9,1,30)
fig,[scores,errs] = s.plot_integr(ddz, "dz", vec0, vec1)

ddz = ddz[:30]
ddx = np.sqrt(1-ddz**2)
pdf = 2*np.pi*ddz * np.exp(-(ddx/sdx)**2/2)/(sdx*sdy*(2*np.pi))
plt.plot(ddz, pdf, zorder=5, label="True")
plt.show()


# Resamplear muestras

In [ ]:
s.save()
!ksource resample "samples_source.txt" -o "resampled" -n 1E5
mcplfile = "resampled.mcpl.gz"

### Vuelva al inicio para analizar distribucion de muestras resampleadas